**# 自动求导**

* 假设我们想对$y=2X^TX$关于列向量$X$求导

In [37]:
import torch
x = torch.arange(4.0)
x 

tensor([0., 1., 2., 3.])

* 在我们计算$y$关于$X$的梯度之前，需要一个地方来存储梯度

In [38]:
x.requires_grad_(True)   # 等价于 x = torch.arange(4.0, requires_grad_(True))
x.grad

* 计算$y$

In [39]:
y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

* 通过调用反向传播函数来自动计算$y$关于$X$的每个分量的梯度

In [40]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

* 计算$x$的另一个函数

In [41]:
# 在默认情况下，Pytorch会累积梯度，需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

* 深度学习中，绝大多数情况$y$都是标量，很少会对一个向量的函数来求导，即目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [42]:
# 对非标量调用 backward 需要传入一个 gradient 参数
x.grad.zero_()
y = x * x  
y.sum().backward()  # 将y转化为标量
# 等价于 y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

* 将某些计算移动到记录的计算图之外

In [43]:
x.grad.zero_()
y = x * x
u = y.detach()   # u 不再是一个关于x的函数，而是一个常数量
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [44]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

* 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），仍然可以计算得到变量的梯度

In [45]:
def f(a):
  b = a * 2
  while b.norm() <1000:
    b = b * 2
  if b.sum() > 0:
    c = b
  else:
    c = 100 * b
  return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d/a

tensor(True)